In [3]:
import torch
import numpy as np
import random
random.seed(1)
torch.cuda.manual_seed(1)
torch.manual_seed(1)
np.random.seed(1)
from NeuralPITF import TimeAttentionPITF, SinglePITF_Loss, DataSet
from torch.autograd import Variable
# from torch.utils import data
import torch.optim as optim
import datetime

In [11]:
train_data_path = 'data/movielens/all_id_core3_train'
test_data_path = 'data/movielens/all_id_core3_test'

movielens_all = np.genfromtxt(train_data_path, delimiter='\t', dtype=float)
movielens_all[:, -1] = (movielens_all[:, -1] - 1135429431000) / (24*3600*1000)
movielens = movielens_all.astype(int)

movielens_test_all = np.genfromtxt(test_data_path, delimiter='\t', dtype=float)
movielens_test_all[:, -1] = (movielens_test_all[:, -1] - 1135429431000) / (24*3600*1000)
movielens_test = movielens_test_all.astype(int)
user_vecs_path = 'PreVecs/movielens/UserVecs.txt'
item_vecs_path = 'PreVecs/movielens/ItemVecs.txt'
tag_user_vec_path = 'PreVecs/movielens/UserTagVecs.txt'
tag_item_vec_path = 'PreVecs/movielens/ItemTagVecs.txt'

def handle_pre_vecs(file_path):
    pre_vecs = np.genfromtxt(file_path, delimiter='\t', dtype=str)
    a = pre_vecs[:, 0]
    b = pre_vecs[:, -1]
    for i in range(len(a)):
        a[i] = a[i].replace('[', '')        
    for i in range(len(b)):
        b[i] = b[i].replace(']', '')   
    pre_vecs[:, 0] = a
    pre_vecs[:, -1] = b  
    pre_vecs = pre_vecs.astype(float)
    return torch.FloatTensor(pre_vecs)


user_vecs = handle_pre_vecs(user_vecs_path)
item_vecs = handle_pre_vecs(item_vecs_path)
tag_user_vecs = handle_pre_vecs(tag_user_vec_path)
tag_item_vecs = handle_pre_vecs(tag_item_vec_path)
ini_embeddings = [user_vecs, item_vecs, tag_user_vecs, tag_item_vecs]

In [23]:
import NeuralPITF
from imp import reload
reload(NeuralPITF)
from NeuralPITF import TimeAttentionPITF, DataSet, SinglePITF_Loss
def train(data, test, m=5, gamma=0.5):
    """
    该函数主要作用： 定义网络；定义数据，定义损失函数和优化器，计算重要指标，开始训练（训练网络，计算在测试集上的指标）
    主要需要调整的参数： m, gamma
    :return:
    """
    # best_file = open('TimeAttention_best_params.txt', 'a')
    learnRate = 0.001
    lam = 0.00005
    dim = 64
    iter_ = 100
    init_st = 0.01
    m = m
    gamma = gamma
    batch_size = 100
    n = 1000
    # 计算numUser, numItem, numTag
    dataload = DataSet(data, test, True)
    num_user, num_item, num_tag = dataload.calc_number_of_dimensions()
    model = TimeAttentionPITF(int(num_user), int(num_item), int(num_tag), dim, init_st, m, gamma, ini_embeddings).cuda()
    # torch.save(model.state_dict(), 'time_attention_initial_params')
    # 对每个正样本进行负采样
    loss_function = SinglePITF_Loss().cuda()
    opti = optim.SGD(model.parameters(), lr=learnRate, weight_decay=lam)
    opti.zero_grad()
    # 每个epoch中的sample包含一个正样本和j个负样本
    best_result = 0
    best_result_state = model.state_dict()
    for epoch in range(iter_):
        # file_ = open('TimeAttentionTureParam.txt', 'a')
        all_data = dataload.get_sequential(num_tag, m, 100)
        losses = []
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        for i, batch in enumerate(dataload.get_batch(all_data, batch_size)):

            # print(batch)
            # input_ = dataload.draw_negative_sample(num_tag, sample, True)
            r = model(torch.LongTensor(batch).cuda())
            opti.zero_grad()
            # print(model.embedding.userVecs.weight)
            loss = loss_function(r)
            # print(loss)
            loss.backward()
            opti.step()
            losses.append(loss.data)
            if i % n == 0:
                print("[%02d/%d] [%03d/%d] mean_loss : %0.2f" % (epoch, iter_, i, len(all_data)/batch_size, np.mean(losses)))
                losses = []
        precision = 0
        recall = 0
        count = 0
        validaTagSet = dataload.validaTagSet
        validaTimeList = dataload.validaUserTimeList
        for u in validaTagSet.keys():
            for i in validaTagSet[u].keys():
                time = validaTimeList[u][i]
                # history_tags = dataload.userShortMemory[u][:m]
                # history_times = dataload.userShortMemory[u][m:]
                number = 0
                tags = validaTagSet[u][i]
                tagsNum = len(tags)
                x_t = torch.LongTensor([u, i]+list(dataload.userShortMemory[u])+[time]).cuda()
                x_t = x_t.unsqueeze(0)
                y_pre = model.predict_top_k(x_t)
                for tag in y_pre[0]:
                    if int(tag) in tags:
                        number += 1
                precision = precision + float(number / 5)
                recall = recall + float(number / tagsNum)
                count += 1
        precision = precision / count
        recall = recall / count
        if precision==0 and recall == 0:
            f_score = 0
        else:
            f_score = 2 * (precision * recall) / (precision + recall)
        print("Precisions: " + str(precision))
        print("Recall: " + str(recall))
        print("F1: " + str(f_score))
        # 将模型最好时的效果保存下来
        if f_score > best_result:
            best_result = f_score
            best_result_state = model.state_dict()
        print("best result: " + str(best_result))
        print("==================================")
        # info = " [%02d/%d] gamma: %f the length m: %d " %(epoch, iter_, gamma, m)   
        # file_.write(info + '\n')
        # file_.write("Precision: " + str(precision) + "  Recall: " + str(recall)+ " F1: " + str(f_score) + " Best Result: " + str(best_result))
        # file_.write('\r\n')
    # torch.save(model, "net.pkl")
    # torch.save(best_result_state, "time_attention_net_params.pkl")
        # file_.close()
    # best_file.write('gamma: %f,  the length: %d, best_result: %f ' %(gamma, m, best_result))
    # best_file.close()
'''
m_params = [2,4,5,6,8,10]
gamma_params = [0.2, 0.4, 0.6, 0.8]
for m in m_params:
    for gamma in gamma_params:
        train(movielens, movielens_test, m, gamma)
'''

train(movielens, movielens_test, 8, 0.6)

Use the BPR for Optimization
2018-10-12 11:01:30
[00/100] [000/29931] mean_loss : 9.06
[00/100] [1000/29931] mean_loss : 1.02
[00/100] [2000/29931] mean_loss : 0.83
[00/100] [3000/29931] mean_loss : 0.59
[00/100] [4000/29931] mean_loss : 0.45
[00/100] [5000/29931] mean_loss : 0.50
[00/100] [6000/29931] mean_loss : 0.38
[00/100] [7000/29931] mean_loss : 0.29
[00/100] [8000/29931] mean_loss : 0.29
[00/100] [9000/29931] mean_loss : 0.22
[00/100] [10000/29931] mean_loss : 0.19
[00/100] [11000/29931] mean_loss : 0.19
[00/100] [12000/29931] mean_loss : 0.18
[00/100] [13000/29931] mean_loss : 0.15
[00/100] [14000/29931] mean_loss : 0.12
[00/100] [15000/29931] mean_loss : 0.15
[00/100] [16000/29931] mean_loss : 0.15
[00/100] [17000/29931] mean_loss : 0.15
[00/100] [18000/29931] mean_loss : 0.16
[00/100] [19000/29931] mean_loss : 0.16
[00/100] [20000/29931] mean_loss : 0.19
[00/100] [21000/29931] mean_loss : 0.18
[00/100] [22000/29931] mean_loss : 0.14
[00/100] [23000/29931] mean_loss : 0.15
[0

[06/100] [000/29931] mean_loss : 0.01
[06/100] [1000/29931] mean_loss : 0.04
[06/100] [2000/29931] mean_loss : 0.05
[06/100] [3000/29931] mean_loss : 0.04
[06/100] [4000/29931] mean_loss : 0.04
[06/100] [5000/29931] mean_loss : 0.04
[06/100] [6000/29931] mean_loss : 0.04
[06/100] [7000/29931] mean_loss : 0.04
[06/100] [8000/29931] mean_loss : 0.05
[06/100] [9000/29931] mean_loss : 0.04
[06/100] [10000/29931] mean_loss : 0.06
[06/100] [11000/29931] mean_loss : 0.05
[06/100] [12000/29931] mean_loss : 0.05
[06/100] [13000/29931] mean_loss : 0.04
[06/100] [14000/29931] mean_loss : 0.04
[06/100] [15000/29931] mean_loss : 0.05
[06/100] [16000/29931] mean_loss : 0.05
[06/100] [17000/29931] mean_loss : 0.04
[06/100] [18000/29931] mean_loss : 0.04
[06/100] [19000/29931] mean_loss : 0.04
[06/100] [20000/29931] mean_loss : 0.05
[06/100] [21000/29931] mean_loss : 0.04
[06/100] [22000/29931] mean_loss : 0.05
[06/100] [23000/29931] mean_loss : 0.05
[06/100] [24000/29931] mean_loss : 0.05
[06/100] [2

[12/100] [2000/29931] mean_loss : 0.03
[12/100] [3000/29931] mean_loss : 0.03
[12/100] [4000/29931] mean_loss : 0.03
[12/100] [5000/29931] mean_loss : 0.03
[12/100] [6000/29931] mean_loss : 0.03
[12/100] [7000/29931] mean_loss : 0.03
[12/100] [8000/29931] mean_loss : 0.04
[12/100] [9000/29931] mean_loss : 0.04
[12/100] [10000/29931] mean_loss : 0.04
[12/100] [11000/29931] mean_loss : 0.04
[12/100] [12000/29931] mean_loss : 0.04
[12/100] [13000/29931] mean_loss : 0.04
[12/100] [14000/29931] mean_loss : 0.04
[12/100] [15000/29931] mean_loss : 0.04
[12/100] [16000/29931] mean_loss : 0.04
[12/100] [17000/29931] mean_loss : 0.04
[12/100] [18000/29931] mean_loss : 0.04
[12/100] [19000/29931] mean_loss : 0.04
[12/100] [20000/29931] mean_loss : 0.04
[12/100] [21000/29931] mean_loss : 0.03
[12/100] [22000/29931] mean_loss : 0.04
[12/100] [23000/29931] mean_loss : 0.05
[12/100] [24000/29931] mean_loss : 0.05
[12/100] [25000/29931] mean_loss : 0.05
[12/100] [26000/29931] mean_loss : 0.05
[12/100]

[18/100] [4000/29931] mean_loss : 0.03
[18/100] [5000/29931] mean_loss : 0.03
[18/100] [6000/29931] mean_loss : 0.03
[18/100] [7000/29931] mean_loss : 0.04
[18/100] [8000/29931] mean_loss : 0.03
[18/100] [9000/29931] mean_loss : 0.04
[18/100] [10000/29931] mean_loss : 0.04
[18/100] [11000/29931] mean_loss : 0.04
[18/100] [12000/29931] mean_loss : 0.04
[18/100] [13000/29931] mean_loss : 0.04
[18/100] [14000/29931] mean_loss : 0.04
[18/100] [15000/29931] mean_loss : 0.04
[18/100] [16000/29931] mean_loss : 0.04
[18/100] [17000/29931] mean_loss : 0.04
[18/100] [18000/29931] mean_loss : 0.04
[18/100] [19000/29931] mean_loss : 0.04
[18/100] [20000/29931] mean_loss : 0.04
[18/100] [21000/29931] mean_loss : 0.03
[18/100] [22000/29931] mean_loss : 0.04
[18/100] [23000/29931] mean_loss : 0.04
[18/100] [24000/29931] mean_loss : 0.05
[18/100] [25000/29931] mean_loss : 0.04
[18/100] [26000/29931] mean_loss : 0.05
[18/100] [27000/29931] mean_loss : 0.04
[18/100] [28000/29931] mean_loss : 0.05
[18/10

[24/100] [6000/29931] mean_loss : 0.03
[24/100] [7000/29931] mean_loss : 0.03
[24/100] [8000/29931] mean_loss : 0.03
[24/100] [9000/29931] mean_loss : 0.03
[24/100] [10000/29931] mean_loss : 0.04
[24/100] [11000/29931] mean_loss : 0.04
[24/100] [12000/29931] mean_loss : 0.04
[24/100] [13000/29931] mean_loss : 0.04
[24/100] [14000/29931] mean_loss : 0.04
[24/100] [15000/29931] mean_loss : 0.03
[24/100] [16000/29931] mean_loss : 0.04
[24/100] [17000/29931] mean_loss : 0.04
[24/100] [18000/29931] mean_loss : 0.03
[24/100] [19000/29931] mean_loss : 0.04
[24/100] [20000/29931] mean_loss : 0.04
[24/100] [21000/29931] mean_loss : 0.04
[24/100] [22000/29931] mean_loss : 0.04
[24/100] [23000/29931] mean_loss : 0.04
[24/100] [24000/29931] mean_loss : 0.04
[24/100] [25000/29931] mean_loss : 0.04
[24/100] [26000/29931] mean_loss : 0.04
[24/100] [27000/29931] mean_loss : 0.05
[24/100] [28000/29931] mean_loss : 0.04
[24/100] [29000/29931] mean_loss : 0.04
Precisions: 0.13445121951219605
Recall: 0.52

...

[72/100] [22000/29931] mean_loss : 0.04
[72/100] [23000/29931] mean_loss : 0.05
[72/100] [24000/29931] mean_loss : 0.05
[72/100] [25000/29931] mean_loss : 0.05
[72/100] [26000/29931] mean_loss : 0.05
[72/100] [27000/29931] mean_loss : 0.05
[72/100] [28000/29931] mean_loss : 0.05
[72/100] [29000/29931] mean_loss : 0.06
Precisions: 0.13201219512195211
Recall: 0.5240926248548201
F1: 0.2109011113704665
best result: 0.2192555532863689
2018-10-12 12:58:28
[73/100] [000/29931] mean_loss : 0.01
[73/100] [1000/29931] mean_loss : 0.02
[73/100] [2000/29931] mean_loss : 0.03
[73/100] [3000/29931] mean_loss : 0.03
[73/100] [4000/29931] mean_loss : 0.03
[73/100] [5000/29931] mean_loss : 0.03
[73/100] [6000/29931] mean_loss : 0.03
[73/100] [7000/29931] mean_loss : 0.03
[73/100] [8000/29931] mean_loss : 0.03
[73/100] [9000/29931] mean_loss : 0.04
[73/100] [10000/29931] mean_loss : 0.04
[73/100] [11000/29931] mean_loss : 0.05
[73/100] [12000/29931] mean_loss : 0.05
[73/100] [13000/29931] mean_loss : 0.

[78/100] [24000/29931] mean_loss : 0.05
[78/100] [25000/29931] mean_loss : 0.05
[78/100] [26000/29931] mean_loss : 0.05
[78/100] [27000/29931] mean_loss : 0.05
[78/100] [28000/29931] mean_loss : 0.05
[78/100] [29000/29931] mean_loss : 0.05
Precisions: 0.13170731707317163
Recall: 0.5214249419279908
F1: 0.21029578377094907
best result: 0.2192555532863689
2018-10-12 13:08:01
[79/100] [000/29931] mean_loss : 0.01
[79/100] [1000/29931] mean_loss : 0.02
[79/100] [2000/29931] mean_loss : 0.03
[79/100] [3000/29931] mean_loss : 0.03
[79/100] [4000/29931] mean_loss : 0.03
[79/100] [5000/29931] mean_loss : 0.04
[79/100] [6000/29931] mean_loss : 0.03
[79/100] [7000/29931] mean_loss : 0.03
[79/100] [8000/29931] mean_loss : 0.03
[79/100] [9000/29931] mean_loss : 0.04
[79/100] [10000/29931] mean_loss : 0.05
[79/100] [11000/29931] mean_loss : 0.04
[79/100] [12000/29931] mean_loss : 0.05
[79/100] [13000/29931] mean_loss : 0.05
[79/100] [14000/29931] mean_loss : 0.05
[79/100] [15000/29931] mean_loss : 0

[84/100] [25000/29931] mean_loss : 0.06
[84/100] [26000/29931] mean_loss : 0.06
[84/100] [27000/29931] mean_loss : 0.05
[84/100] [28000/29931] mean_loss : 0.05
[84/100] [29000/29931] mean_loss : 0.05
Precisions: 0.13048780487804967
Recall: 0.518376161440186
F1: 0.20849290735391038
best result: 0.2192555532863689
2018-10-12 13:17:46
[85/100] [000/29931] mean_loss : 0.00
[85/100] [1000/29931] mean_loss : 0.03
[85/100] [2000/29931] mean_loss : 0.03
[85/100] [3000/29931] mean_loss : 0.03
[85/100] [4000/29931] mean_loss : 0.03
[85/100] [5000/29931] mean_loss : 0.03
[85/100] [6000/29931] mean_loss : 0.03
[85/100] [7000/29931] mean_loss : 0.03
[85/100] [8000/29931] mean_loss : 0.03
[85/100] [9000/29931] mean_loss : 0.04
[85/100] [10000/29931] mean_loss : 0.05
[85/100] [11000/29931] mean_loss : 0.05
[85/100] [12000/29931] mean_loss : 0.05
[85/100] [13000/29931] mean_loss : 0.05
[85/100] [14000/29931] mean_loss : 0.05
[85/100] [15000/29931] mean_loss : 0.05
[85/100] [16000/29931] mean_loss : 0.

[90/100] [27000/29931] mean_loss : 0.05
[90/100] [28000/29931] mean_loss : 0.05
[90/100] [29000/29931] mean_loss : 0.06
Precisions: 0.13109756097561065
Recall: 0.5223141695702672
F1: 0.2095894839122598
best result: 0.2192555532863689
2018-10-12 13:27:31
[91/100] [000/29931] mean_loss : 0.00
[91/100] [1000/29931] mean_loss : 0.02
[91/100] [2000/29931] mean_loss : 0.03
[91/100] [3000/29931] mean_loss : 0.03
[91/100] [4000/29931] mean_loss : 0.03
[91/100] [5000/29931] mean_loss : 0.03
[91/100] [6000/29931] mean_loss : 0.03
[91/100] [7000/29931] mean_loss : 0.03
[91/100] [8000/29931] mean_loss : 0.03
[91/100] [9000/29931] mean_loss : 0.04
[91/100] [10000/29931] mean_loss : 0.05
[91/100] [11000/29931] mean_loss : 0.05
[91/100] [12000/29931] mean_loss : 0.05
[91/100] [13000/29931] mean_loss : 0.05
[91/100] [14000/29931] mean_loss : 0.05
[91/100] [15000/29931] mean_loss : 0.05
[91/100] [16000/29931] mean_loss : 0.05
[91/100] [17000/29931] mean_loss : 0.05
[91/100] [18000/29931] mean_loss : 0.

[96/100] [29000/29931] mean_loss : 0.06
Precisions: 0.13170731707317163
Recall: 0.522822299651568
F1: 0.21040918740302766
best result: 0.2192555532863689
2018-10-12 13:37:01
[97/100] [000/29931] mean_loss : 0.01
[97/100] [1000/29931] mean_loss : 0.02
[97/100] [2000/29931] mean_loss : 0.03
[97/100] [3000/29931] mean_loss : 0.03
[97/100] [4000/29931] mean_loss : 0.03
[97/100] [5000/29931] mean_loss : 0.03
[97/100] [6000/29931] mean_loss : 0.04
[97/100] [7000/29931] mean_loss : 0.04
[97/100] [8000/29931] mean_loss : 0.04
[97/100] [9000/29931] mean_loss : 0.04
[97/100] [10000/29931] mean_loss : 0.05
[97/100] [11000/29931] mean_loss : 0.05
[97/100] [12000/29931] mean_loss : 0.05
[97/100] [13000/29931] mean_loss : 0.05
[97/100] [14000/29931] mean_loss : 0.05
[97/100] [15000/29931] mean_loss : 0.06
[97/100] [16000/29931] mean_loss : 0.05
[97/100] [17000/29931] mean_loss : 0.05
[97/100] [18000/29931] mean_loss : 0.05
[97/100] [19000/29931] mean_loss : 0.05
[97/100] [20000/29931] mean_loss : 0.